In [1]:
from __future__ import print_function

import numpy as np
import warnings

from keras.layers import merge, Input, Lambda, Layer, RepeatVector,Merge
from keras.layers import Dense, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model, Sequential
from keras.preprocessing import image
import keras.backend as K
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file



Using TensorFlow backend.


In [22]:

def shakemerge(x1,x2):
    def mul(xa):
        return xa[0] * xa[1]
    def mul_shape(xa_shape):
        return xa_shape[0]
    batch_size = K.shape(x1)[0]
    #alpha = K.random_uniform((batch_size,1,1,1),0,1)
    alpha = Lambda(lambda x:K.random_uniform((K.shape(x)[0],1,1,1), 0, 1))(x1)
    x1a = merge([x1,alpha], mode=mul, output_shape=mul_shape)
    x2a = merge([x2,1-alpha], mode=mul, output_shape=mul_shape)
    return merge([x1a,x2a], mode='sum')
    


In [23]:
x1 = Input(shape=(3,5,5,))
x2 = Input(shape=(3,5,5,))
out = shakemerge(x1,x2)
model1 = Model([x1,x2], out)

TypeError: Output tensors to a Model must be Keras tensors. Found: Tensor("add_3:0", shape=(?, 3, 5, 5), dtype=float32)

In [12]:
out

<tf.Tensor 'add_2:0' shape=(?, 3, 5, 5) dtype=float32>

In [37]:
def asum(x1,x2):
    def mul(xa):
        return xa[0] * xa[1]
    def mul_shape(xa_shape):
        return xa_shape[0]
    alpha = Lambda(lambda x:K.random_uniform((K.shape(x)[0],1,1), 0, 1))(x1)
    xa1 = merge([x1,alpha], mode=mul, output_shape=mul_shape)
    xa2 = merge([x2,alpha], mode=mul, output_shape=mul_shape)
    xa12 = merge([xa1,xa2], mode='sum')
    return xa12


x1 = Input(shape=(3,5,))
x2 = Input(shape=(3,5,))
xa12 = asum(x1,x2)
model = Model([x1,x2], xa12)

In [38]:
X1 = np.ones((4,3,5))
X2 = 4* np.ones((4,3,5))
model.predict([X1,X2])

array([[[ 4.65165329,  4.65165329,  4.65165329,  4.65165329,  4.65165329],
        [ 4.65165329,  4.65165329,  4.65165329,  4.65165329,  4.65165329],
        [ 4.65165329,  4.65165329,  4.65165329,  4.65165329,  4.65165329]],

       [[ 0.61420619,  0.61420619,  0.61420619,  0.61420619,  0.61420619],
        [ 0.61420619,  0.61420619,  0.61420619,  0.61420619,  0.61420619],
        [ 0.61420619,  0.61420619,  0.61420619,  0.61420619,  0.61420619]],

       [[ 0.78121781,  0.78121781,  0.78121781,  0.78121781,  0.78121781],
        [ 0.78121781,  0.78121781,  0.78121781,  0.78121781,  0.78121781],
        [ 0.78121781,  0.78121781,  0.78121781,  0.78121781,  0.78121781]],

       [[ 4.92272615,  4.92272615,  4.92272615,  4.92272615,  4.92272615],
        [ 4.92272615,  4.92272615,  4.92272615,  4.92272615,  4.92272615],
        [ 4.92272615,  4.92272615,  4.92272615,  4.92272615,  4.92272615]]], dtype=float32)

In [ ]:
x1 = Input(shape=(15,200,))
x2 = Input(shape=(15,))
x12 = K.dot(x1,x2)
#model1 = Model([x1,x2], x12)

In [ ]:
x1

In [ ]:
x

In [ ]:
input_layer = Input(shape=(5,5,))
x = input_layer
s0 = K.shape(x)[0]
s1 = 5
s2 = 5
r = Lambda(lambda x:K.random_uniform((s0,1,1), 0, 1))(x)
r = Lambda(lambda x:K.repeat_elements(x,s1,axis=1))(r)
r = Lambda(lambda x:K.repeat_elements(x,s2,axis=2))(r)
xr = merge([x,r],mode='mul')
model1 = Model(input_layer, xr)

In [ ]:
s1

In [ ]:
model2 = Model(input_layer, r)

In [ ]:
X = np.random.random((4,5,5))
model2.predict(X)

In [ ]:
input_layer = Input(shape=(5,5,))

In [ ]:
 Lambda(lambda x:K.random_uniform((K.shape(x)[0],1), 0, 1))(x)

In [ ]:
input_layer = Input(shape=(5,5,))
x = Lambda(lambda x:K.batch_flatten(x))(input_layer)
r = Lambda(lambda x:K.random_uniform((K.shape(x)[0],), 0, 1))(x)
r = Lambda(lambda x:K.expand_dims(x,dim=1))(r)
r = RepeatVector(5)(r)

y = Lambda(lambda x:x * r)(input_layer)

In [ ]:
input_layer

In [ ]:
y

In [ ]:
model1 = Model(input_layer, r)

In [ ]:
X = np.random.random((4,5,5))

In [ ]:
model1.predict(X)

In [ ]:
input_layer = Input(shape=(5,))
x1 = Dense(32)(input_layer)
x1 = Dense(3)(x1)
x2 = Dense(32)(input_layer)
x2 = Dense(3)(x2)
alpha = K.random_uniform((1,1),0,1)[0]
#alpha = K.reshape(alpha, K.shape(x1))
y1 = (Lambda(lambda x:x*alpha ))(x1)
y2 = (Lambda(lambda x:x*(1-alpha) ))(x2)
y = merge([y1,y2], mode='sum')

In [ ]:
model1 = Model(input_layer, y)

In [ ]:
X = np.random.random((4,5))

In [ ]:
Y1 = model1.predict(X)

In [ ]:
Y1

In [ ]:
class RandomAffine(Layer):

    def __init__(self, **kwargs):
        super(RandomAffine, self).__init__(**kwargs)

    def call(self, inputs, mask=None):
        x,y = inputs[0],inputs[1]
        alpha = K.random_normal(K.shape(x),0,1)
        return alpha * x + (1-alpha) * y


In [ ]:
input_layer = Input(shape=(5,))
x1 = Dense(32)(input_layer)
x1 = Dense(3)(x1)
x2 = Dense(32)(input_layer)
x2 = Dense(3)(x2)
alpha = K.random_uniform(K.shape(x1),0,1)

In [ ]:
K.sum(x2,axis=-1)

In [ ]:
K.sum(x2)

In [ ]:
alpha

In [ ]:
model1 = Model(input_layer, alpha)